In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from pandas import read_csv
import pyodbc
from sqlalchemy import create_engine,text


In [2]:
data = read_csv("2021-all-meter.csv",index_col=0)

In [3]:
data.head()

,AMI Meter ID,date,16335457,18762999,230156289,56068032,56073008,56210792,56464971,66675284,...,98828750,98923621,98923641,98923648,98923655,98923661,98923662,98923663,Residential,Total
0,2021-03-21 00:00:00,2021-03-21 00:30:00,104.192,28.649,46.68,13.104,9.195,373.44,19.656,5.491,...,20.640,0.324,15.228,16.212,47.430,51.60,16.35,21.624,67808.216,71482.267
1,2021-03-21 00:30:00,2021-03-21 01:00:00,104.848,28.578,51.60,15.728,9.810,373.44,20.640,6.278,...,21.624,0.324,14.736,15.228,39.198,46.68,16.35,17.688,67808.216,71395.502
2,2021-03-21 01:00:00,2021-03-21 01:30:00,106.160,29.929,55.29,14.416,8.580,375.96,19.656,7.853,...,9.338,0.160,15.228,14.244,40.976,44.22,14.30,18.180,68586.253,72266.724
3,2021-03-21 01:30:00,2021-03-21 02:00:00,104.192,29.488,50.37,16.384,11.040,378.36,18.672,6.278,...,9.418,0.324,14.736,14.244,42.165,44.22,16.35,16.704,68586.253,72069.140
4,2021-03-21 02:00:00,2021-03-21 02:30:00,104.192,28.645,52.83,15.728,12.270,375.96,20.640,4.704,...,8.928,0.324,15.228,16.212,50.782,46.68,18.40,18.180,71152.560,74632.641


In [4]:
ar = []
for i in range(0,data.shape[0],1):
    for c in data.select_dtypes(include=np.number).columns:
        ar.append([c,data.iloc[[i]]["AMI Meter ID"].values[0], data.iloc[[i]]["date"].values[0], int(data.iloc[[i]][c].values)])

SQLData = pd.DataFrame(ar)


In [5]:
SQLData.rename(columns = {0:"meter_name", 1:"start_time",2:"end_time", 3:"value"}, inplace = True)

In [6]:
SQLData.head()

,meter_name,start_time,end_time,value
0,16335457,2021-03-21 00:00:00,2021-03-21 00:30:00,104
1,18762999,2021-03-21 00:00:00,2021-03-21 00:30:00,28
2,230156289,2021-03-21 00:00:00,2021-03-21 00:30:00,46
3,56068032,2021-03-21 00:00:00,2021-03-21 00:30:00,13
4,56073008,2021-03-21 00:00:00,2021-03-21 00:30:00,9


In [7]:
SQLData.shape

(445200, 4)

In [8]:
# IMPORT THE SQALCHEMY LIBRARY's CREATE_ENGINE METHOD


# DEFINE THE DATABASE CREDENTIALS
user = 'root'
password = 'NewPassword'
host = '127.0.0.1'
port = 3306
database = 'demand_response_portal'

# PYTHON FUNCTION TO CONNECT TO THE MYSQL DATABASE AND
# RETURN THE SQLACHEMY ENGINE OBJECT
def get_connection():
    return create_engine(
        url="mysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

#pyodbc.
#if __name__ == '__main__':
engine = []

# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
engine = get_connection()
conn = engine.connect()
SQLData.to_sql('meter_data', con=engine, if_exists='replace')



445200

In [9]:
query = text("select meter_data.meter_name, meter_data.value from meter_data where meter_data.meter_name = 'Total'")
GetSQLTest = pd.read_sql_query(query,conn)

In [10]:
GetSQLTest["value"].count()

8400